In [12]:
import pandas as pd
import os
import numpy as np
import sys

In [64]:
module_path = "/data/luojaa/eukgen/"
if module_path not in sys.path:
    sys.path.append(module_path)
import altair as alt
from core_functions.altair_plots import plot_cumsum_counts
%run /data/luojaa/eukgen/styling_and_visualisation.py

In [23]:
remove_target = set("""WP_148309537_1
WP_012149064_1
WP_012149064_1
WP_012149064_1
WP_002955443_1
WP_161914295_1
WP_074631236_1
WP_012149064_1
WP_071002269_1
WP_071002269_1
WP_012149064_1
WP_074631236_1
WP_012149064_1
WP_161914295_1
WP_002955443_1
WP_015716723_1
WP_087019688_1
WP_014659467_1
WP_014659467_1
WP_023652245_1
WP_070236922_1
WP_014659467_1
WP_143332899_1
WP_015903833_1
WP_038561231_1
UDQ99485_1
WP_038561231_1
WP_014659467_1
WP_194398583_1
WP_012149064_1""".split("\n"))

In [29]:
asgard_annot = pd.read_csv("/data/luojaa/kegg_hmm_db/search_asgard/asgard_hhblits_results.cut.tsv", sep = "\t")
asgard_annot_hmm = asgard_annot[(asgard_annot.Query != asgard_annot.Target)].reset_index()
asgard_annot_hmm = asgard_annot_hmm[(asgard_annot_hmm.Pairwise_cov > .8) & (asgard_annot_hmm.Prob > 80)]

In [31]:
prok_hh2mm = pd.read_csv("/data/luojaa/eukgen/microcosm/prok2111_mm2hh_map.tsv", sep = "\t", header = None).set_index(1).to_dict()[0]
asgard_hh2mm = pd.read_csv("/data/luojaa/eukgen/microcosm/asgards_mm2hh_map.tsv", sep = "\t", header = None).set_index(1).to_dict()[0]

In [32]:
#asgard_annot = asgard_annot[~asgard_annot.Target.isin(remove_target)]

In [34]:
asgard_annot_hh2mm = pd.concat([asgard_annot_hmm["Query"].apply(lambda x: asgard_hh2mm[x]), asgard_annot_hmm["Target"].apply(lambda x: prok_hh2mm[x])], axis = 1)

In [36]:
asgard_clusters =  pd.read_csv("/data/luojaa/eukgen/processing/asgard2023/asgard2023_clean.repseq.cascaded_cluster.tsv", sep = "\t", header = None)
asgard_rep2members = asgard_clusters.groupby(0)[1].apply(list).to_dict()

prok_clusters =  pd.read_csv("/data/luojaa/eukgen/processing/prok2111/prok2111.repseq.cascaded_cluster.tsv", sep = "\t", header = None)
prok_rep2members = prok_clusters.groupby(0)[1].apply(list).to_dict()

In [37]:
len(asgard_annot_hh2mm.Query.unique())

3140

In [38]:
len(asgard_annot_hh2mm.Target.unique())

11663

In [69]:
accessions_reps = asgard_annot_hh2mm.loc[:,["Query", "Target"]].groupby("Query")["Target"].apply(list).reset_index()
accessions_reps["asgards"] = accessions_reps.Query.apply(lambda x: "\n".join(asgard_rep2members[x]))
accessions_reps["proks"] = accessions_reps.Target.apply(lambda x: "\n".join(["\n".join(prok_rep2members[i]) for i in x]))
accessions_reps["Query_count"] = accessions_reps.asgards.apply(lambda x: len(x.split('\n')))
accessions_reps["Target_count"] = accessions_reps.proks.apply(lambda x: len(x.split('\n')))
accessions_reps["Target_rep_count"] = accessions_reps.Target.apply(lambda x: len(x))

In [78]:
accessions_outdf = accessions_reps[accessions_reps.Target_count > 5]

In [79]:
accessions_outdf

,Query,Target,asgards,proks,Query_count,Target_count,Target_rep_count
2,CP070658.1_1228,[WP_129888408_1],CP070658.1_1228\nPJET01000016.1_2\nJAEOUF01000...,WP_129888408_1\nWP_119884262_1\nWP_040373171_1...,5,47,1
3,CP070658.1_1286,"[ABX07096_1, WP_187108689_1, WP_010878997_1, W...",CP070658.1_1286\nJAEOTZ010000050.1_1\nJAHQWI01...,ABX07096_1\nWP_035192285_1\nWP_097113660_1\nWP...,64,1596,8
4,CP070658.1_1783,[WP_011762996_1],CP070658.1_1783\nUYP47339.1\nTKJ19827.1\nJYIM0...,WP_011762996_1\nWP_013679112_1\nWP_048062892_1...,5,228,1
5,CP070658.1_2539,"[WP_013719373_1, WP_015018353_1]",CP070658.1_2539\nPRDJ01000077.1_30\nJAEOUF0100...,WP_013719373_1\nWP_007140539_1\nWP_011020689_1...,18,564,2
6,CP070658.1_61,"[WP_012681683_1, WP_012860632_1, WP_013606876_...",CP070658.1_61\nJAHKKT010000122.1_3\nJAHQXB0100...,WP_012681683_1\nWP_053333919_1\nWP_171226072_1...,48,31849,28
...,...,...,...,...,...,...,...
3135,WP_162306655.1,"[WP_162306655_1, WP_003755061_1, WP_077140017_...",WP_162306655.1\nQMYW01000229.1_5\nUYP47954.1\n...,WP_162306655_1\nWP_157750288_1\nWP_223935243_1...,4,11968,8
3136,WP_162306670.1,"[WP_147664079_1, WP_071932562_1]",WP_162306670.1\nWP_147664079.1\nUYP46893.1\nKK...,WP_147664079_1\nWP_163301122_1\nWP_136796325_1...,6,144,2
3137,WP_162306801.1,"[WP_083102639_1, UDQ97857_1]",WP_162306801.1\nJAHQWR010000039.1_8,WP_083102639_1\nWP_082379229_1\nWP_122979810_1...,2,898,2
3138,WP_244626339.1,"[WP_135081493_1, WP_077339326_1, AFS48230_1, W...",WP_244626339.1\nTFG08889.1\nNHJ20058.1\nJAEORW...,WP_135081493_1\nWP_038640406_1\nWP_011495736_1...,5,29060,59


In [80]:
len(accessions_outdf.Query.unique())

2802

In [81]:
accessions_outdf.to_csv("accessions_outdf.microcosm_asgards.tsv", sep = "\t", index = None)

In [82]:
plot1, data1 = plot_cumsum_counts(accessions_outdf.Query_count, title='number asgard members', plot_type='default')
plot2, data2 = plot_cumsum_counts(accessions_outdf.Target_count, title='number prok members', plot_type='default')
all_data = pd.concat([data1, data2])
plot, data = plot_cumsum_counts(all_data, formatted_data=True, title='microcosm size distributions', plot_type='default', x_label = "Number of proteins")
plot

alt.LayerChart(...)

In [83]:
ids = list(accessions_outdf["Query"])
asgards = list(accessions_outdf["asgards"])
proks = list(accessions_outdf["proks"])

In [84]:
outpathroot = "/data/luojaa/eukgen/processing/microcosm_asgards/"

In [85]:
# write accessions files
for i in range(len(accessions_outdf)):
    fid, queries, targets = ids[i], asgards[i], proks[i]
    os.system(f"mkdir {outpathroot}{fid}")
    outpath_q = f"{outpathroot}{fid}/{fid}.query.acc"
    outpath_t = f"{outpathroot}{fid}/{fid}.target.acc"
    with open(outpath_q, "w") as outfile:
        print(queries, file=outfile)
    with open(outpath_t, "w") as outfile:
        print(targets, file=outfile)

In [86]:
# write swarm file to run microcosms
with open("../processing/run_microcosm_asgards.swarm", "w") as outfile:
    for fid in os.listdir(outpathroot):
        line = f"source myconda; python -u /data/luojaa/eukgen/microcosm_analysis.py --file_root {outpathroot}{fid}/ --file_basename {fid} --delete_intermediate_files"
        print(line,file=outfile)